# Homework 3, BEE 6940 (Due By 4/13/23, 9:00PM)

**Name**:

**ID**:

## Overview

### Instructions

- Problem 1 asks you to calibrate the Rahmstorf (2007) semi-empirical sea-level rise model using Markov chain Monte Carlo and `Turing.jl`.
- Problem 2 asks you to generate projections of global sea-level rise based on this model for different Representative Concentration Pathways.
- Problem 3 asks you to compare the results of this calibration with your bootstrap estimates from Lab 5.

### Load Environment

The following code loads the environment and makes sure all needed packages are installed. This should be at the start of most Julia scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Teaching/BEE6940/hw/hw3`


In [2]:
using Random
using CSVFiles # load CSV data
using DataFrames # data storage and presentation
using Plots # plotting library
using StatsPlots # statistical plotting
using Distributions # statistical distribution interface
using Turing # probabilistic programming and MCMC
using Optim # optimization library


## Problems (100 points)

### Problem 1 (60 points)

In this problem, we will use Markov chain Monte Carlo to calibrate the [Rahmstorf (2007)](https://doi.org/10.1073/pnas.0907765106) semi-empirical sea-level rise model:
$$\frac{dH(t)}{dt} = \alpha (T(t) - T_0),$$
where $H(t)$ is global mean sea-level anomaly (in mm), $T(t)$ is the global mean surface temperature (in $^\circ$ C), $T_0$ is the temperature (in $^\circ$ C) when sea-level rise is in equilibrium, and $\alpha$ is the sea-level rise sensitivity to warming. Discretizing this equation with an annual time step yields
$$H(t+1) = H(t) + \alpha (T(t) - T_0).$$

Our goal in this problem is to calibrate this model and quantify parametric uncertainties using Markov chain Monte Carlo.

In [25]:
function rahmstorf_model(α, T₀, H₀, temp_data)
    temp_effect = α .* (temp_data .- T₀)
    slr_predict = cumsum(temp_effect) .+ H₀
    return slr_predict
end

rahmstorf_model (generic function with 1 method)

First, let's load the data.

In [44]:
# load data files
slr_data = DataFrame(load("data/CSIRO_Recons_gmsl_yr_2015.csv"))
gmt_data = DataFrame(load("data/NOAA_IPCC_RCPtempsscenarios.csv"))

slr_data[:, :Time] = slr_data[:, :Time] .- 0.5; # remove 0.5 from Times
dat = leftjoin(slr_data, gmt_data, on="Time") # join data frames on time

select!(dat, [1, 2, 6])  # drop columns we don't need
first(dat, 6)

┌ Warning: In data/NOAA_IPCC_RCPtempsscenarios.csv line 426 has 0 fields but 13 fields are expected. Skipping row.
└ @ TextParse /Users/vs498/.julia/packages/TextParse/gNKVx/src/csv.jl:382


Row,Time,GMSL (mm),Historical NOAA temp & CNRM RCP 8.5 with respect to 20th century
,Float64,Float64,Float64?
1,1880.0,-158.7,-0.16
2,1881.0,-153.1,-0.1
3,1882.0,-169.9,-0.12
4,1883.0,-164.6,-0.17
5,1884.0,-143.7,-0.23
6,1885.0,-145.2,-0.2


In [27]:
slr_data = dat[:, 2];
temp_data = dat[:,3];

Next, let's specify the statistical model. Denote the residuals as $$z(\mathbf{x}, t)=y_t - F(\mathbf{x}, t),$$ where $y_t$ is the data at time $t$ and $F(\mathbf{x}, t)$ is the model output at time $t$ with parameters $\mathbf{x}$. 

For (relative) simplicity, we will assume $z(\mathbf{x}, t)$ is given as an AR(1) process, *i.e.* $$z(\mathbf{x}, t) = \rho z(\mathbf{x}, t-1) + \varepsilon_t, \quad \varepsilon_t \sim \text{Normal}(0, \sigma^2).$$

The unconditional variance of an AR(1) process is $$\text{Var}(z) = \frac{\sigma^2}{1-\rho^2},$$ so the likelihood for $z$ (abusing notation to only refer to the time-index, not the parameters) can be written as:
$$\begin{align*}
z_1 &\sim \text{Normal}\left(0, \frac{\sigma^2}{1-\rho^2}\right) \\
z_t &\sim \text{Normal}(\rho z_{t-1}, \sigma^2)
\end{align*}$$

#### Problem 1.1 (15 points)

Select priors for each of the parameters $\alpha$, $T_0$, $H_0$, $\rho$, and $\sigma^2$. Justify these priors based on relevant considerations of prior information and model structure and regularization/informativeness. 

#### Problem 1.2 (15 points)

Code the posterior model in `Turing.jl` syntax in the below function. As a tip, do not directly model the residuals (this will increase the model dimension due to how `Turing.jl` parses the model code), but write out the residuals in terms of the data and the model output and rearrange as discussed in class. In other words, use sampling commands like:

```julia
sl_predict = rahmstorf_model(...)
sl_data[1] ~ Normal(sl_predict[1], 2)
```
rather than 
```julia
residuals = rahmstorf_model(...) .- sl_data
residuals[1] ~ Normal(0, 2)
```

In [28]:
@model function slr_posterior(sl_data, temp_data)
    # define priors

    # define likelihood
end

┌ Warning: Model definition seems empty, still continue.
└ @ DynamicPPL /Users/vs498/.julia/packages/DynamicPPL/UFajj/src/compiler.jl:633


slr_posterior (generic function with 2 methods)

#### Problem 1.3 (15 points)

Draw samples from the posterior using MCMC. How did you decide how many chains to use and how many iterations to run the chain(s) for? How did you determine if your Markov chain(s) converged (include any plots and/or quantities which you used to make this assessment)?

Tip: If you find poor convergence or that your sampler is taking a while, look at your model definition. You may need to reconsider your priors, or you may be able to reformulate the model to make it more efficient. Or you might just need to increase the number of iterations, because a starting point took longer to converge.

In [ ]:
chain = let
    model = slr_posterior(slr_data, temp_data) # load the data into the model object
    sampler = NUTS() # use the no-u-turn sampler (there are other options, but this is a good one)
    # replace these with your own values
    n_per_chain = 1 # number of iterations per chain
    nchains = 1 # number of MCMC chains to run
#    sample(model, sampler, MCMCThreads(), n_per_chain, nchains; drop_warmup=true) # uncomment for multiple threads, i.e. nchains > 1
    sample(model, sampler, n_per_chain; drop_warmup=true) # comment out to use multiple chains
end

#### Problem 1.4 (15 points)

Make a corner/pairs plot of the posterior samples using `corner(chain[:, :, 1])` (unfortunately, this plot recipe does not work well with multiple chains, but if your chains have converged and were run long enough, they should all look roughly the same). Do the correlations between the various parameters make sense to you? Why or why not?

### Problem 2 (30 points)

In this problem, we will use our calibrated model above with NOAA's RCP8.5 temperature projections to look at different point estimates of the model parameters and now their associated projections compare to the posterior predictive probability distribution.

#### Problem 2.1 (5 points)

Compute the maximum *a posteriori* (MAP) and maximum likelhood (MLE) estimates for the model.

#### Problem 2.2 (15 points)

Use `gmt_data` (the first column for the years and the fourth column for temperatures) to simulate sea-levels using your model for 1,000 posterior parameter samples, as well as for the MLE and MAP estimates. Plot the 95% quantiles by year for the posterior predictive samples along with the posterior median and the MAP and MLE projections.

#### Problem 2.3 (10 points)

Based on your plot in Problem 2.2, discuss the potential implications for risk assessment of using the different point estimates (MAP and MLE) vs the broader posterior predictive ensemble. When would you suggest that these estimates be used or not used?